In [0]:
!pip install nltk

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/bf/ce/0d0e61429f603bac433910d99ef1a02ce45a8967ffbe3cbee48599e62d88/regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━ 1.0/1.5 MB 29.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/792.7 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 39.1 MB/s eta 0:00:00
Note: you may need to restart

In [0]:
!pip install wordcloud 

In [0]:
import numpy as np
import pandas as pd
 

#------------------------------------Plots
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import StandardScaler,MinMaxScaler


import nltk
nltk.download('stopwords')

from collections import Counter
import re

from nltk.corpus import stopwords




spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
pd.set_option('display.max_columns', 500)


sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(15,10)})

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spark-5cc75cd9-d898-47a9-8222-52/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Global Functions and variables

In [0]:
def plot_chart(
    df,
    chart_type="bar",             # "bar" o "line"
    x_col="x",
    y_col="y",
    text_col=None,
    group_col=None,              # para múltiples líneas o barras
    sort_by=None,
    ascending=True,
    x_label=None,
    y_label=None,
    title="Chart",
    color=None,                  # puede ser columna string o color fijo
    color_sequence=None,         # lista de colores para los grupos
    width=1200,
    height=700,
    text_as_percentage=True,
    text_position="outside",     # "outside" para barras, "top center" para líneas
    barmode="group",
    category_order=None              # solo para barras: "group", "stack"
):
    """
    Función flexible para graficar barras o líneas con agrupación opcional.

    Parámetros:
    - df: DataFrame de entrada
    - chart_type: "bar" o "line"
    - x_col, y_col: columnas para eje X e Y
    - text_col: columna con valores de texto (por ejemplo, porcentaje)
    - group_col: columna de agrupación para múltiples líneas o barras
    - sort_by: columna para ordenar (opcional)
    - ascending: orden ascendente o descendente
    - x_label, y_label: etiquetas personalizadas para los ejes
    - title: título del gráfico
    - color: nombre de la columna para colorear (o color fijo si no hay agrupamiento)
    - color_sequence: lista de colores para los grupos
    - width, height: dimensiones
    - text_as_percentage: si mostrar el texto como porcentaje
    - text_position: posición del texto
    - barmode: modo de barra: "group", "stack", etc.
    - category_order: list with the order that you want for each axes
    """

    df_plot = df.copy()
    if sort_by:
        df_plot = df_plot.sort_values(by=sort_by, ascending=ascending)

    labels = {x_col: x_label or x_col, y_col: y_label or y_col}

    color_args = dict()
    if group_col:
        color_args["color"] = group_col
    elif isinstance(color, str):
        color_args["color"] = color
    else:
        color_args["color_discrete_sequence"] = [color or "#b35151"]

    if color_sequence:
        color_args["color_discrete_sequence"] = color_sequence

    if chart_type == "bar":
        fig = px.bar(
            df_plot,
            x=x_col,
            y=y_col,
            text=text_col if text_col else None,
            labels=labels,
            **color_args
        )
        fig.update_layout(barmode=barmode)

    elif chart_type == "line":
        fig = px.line(
            df_plot,
            x=x_col,
            y=y_col,
            text=text_col if text_col else None,
            labels=labels,
            markers=True,
            **color_args
        )
    else:
        raise ValueError("chart_type debe ser 'bar' o 'line'.")

    # Layout general
    fig.update_layout(
    plot_bgcolor='white',
    xaxis=dict(
        gridcolor='lightgray',
        title=labels[x_col],
        tickfont=dict(size=12, family='Arial', color='black'),
        title_font=dict(size=19, family='Arial', color='black'),
        title_standoff=28,
        categoryorder="array",
        categoryarray=category_order
    ),
    yaxis=dict(
        gridcolor='lightgray',
        title=labels[y_col],
        tickfont=dict(size=14, family='Arial', color='black'),
        title_font=dict(size=19, family='Arial', color='black')
    ),
    width=width,
    height=height,
    titlefont=dict(size=19, family='Arial', color='black'),
    title=title
)


    # Texto sobre barras o puntos
    if text_col:
        fig.update_traces(
            texttemplate='%{text:.0%}' if text_as_percentage else '%{text}',
            textposition=text_position,
            textfont=dict(size=18)
        )

    fig.show()

#--------------------------------------------------------------------------------------------------------------------------------------------


def heatmap_categoricas_vs_numerica(
    df, 
    cat_x, 
    cat_y, 
    num_var, 
    colorscale,
    width,
    height,
    title=None,
    x_title=None,
    y_title=None
):
    """
    Genera un heatmap que muestra la agregación de una variable numérica
    para cada combinación de dos variables categóricas.

    Parámetros:
    - df: DataFrame con los datos.
    - cat_x: nombre columna categórica para eje X.
    - cat_y: nombre columna categórica para eje Y.
    - num_var: nombre columna numérica para agregar.
    - colorscale: escala de colores para el heatmap.
    - width: Width of the figure in pixels.
    - height: Height of the figure in pixels.
    - title: título del gráfico (string).
    """


    
    
    fig = go.Figure(data=go.Heatmap(
    z=df[num_var],
    x=df[cat_x],
    y=df[cat_y],
    colorscale=    colorscale,
    text=df[num_var],                  
    texttemplate="%{text:.2f}",            
    textfont={"size": 12, "color": "white"} , zmin=0, zmax=df[num_var].quantile(0.95)), )
    

    fig.update_layout(
        title=title, width=width, height=height,
        xaxis_title=x_title,
        yaxis_title=y_title
        
    )

    fig.show()



#--------------------------------------------------------------------------------------------------------------------------------------------



def plot_purpose_treemap(df, status_column='approved', purpose_column='purpose'):
    # Separar por estado
    df_approved = df[df[status_column] == 1]
    df_rejected = df[df[status_column] == 0]

    def clean_and_tokenize(text_series):
        text = ' '.join(text_series.dropna().astype(str)).lower()
        text = re.sub(r'\d+', '', text)                      # quitar números
        text = re.sub(r'[^\w\s]', '', text)                  # quitar puntuación
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'-', ' ', text).strip()    
        text = re.sub(r'_', ' ', text).strip()             
        tokens = text.split()
        tokens = [word for word in tokens if word not in custom_stopwords]
        return tokens

    # Tokenización
    approved_tokens = clean_and_tokenize(df_approved[purpose_column])
    rejected_tokens = clean_and_tokenize(df_rejected[purpose_column])

    # Top palabras
    top_approved = Counter(approved_tokens).most_common(20)
    top_rejected = Counter(rejected_tokens).most_common(20)

    # DataFrames
    df_app = pd.DataFrame(top_approved, columns=['word', 'raw_count'])
    df_app['status'] = 'Approved'

    df_rej = pd.DataFrame(top_rejected, columns=['word', 'raw_count'])
    df_rej['status'] = 'Rejected'

    # Normalizar los conteos (suma 100 en cada grupo)
    df_app['count'] = df_app['raw_count'] / df_app['raw_count'].sum() * 100
    df_rej['count'] = df_rej['raw_count'] / df_rej['raw_count'].sum() * 100

    # Concatenar
    df_all = pd.concat([df_app, df_rej], ignore_index=True)

    # Texto personalizado con porcentaje
    df_all['label'] = df_all['word'] + '<br>' + df_all['count'].round(1).astype(str) + '%'

    # Treemap
    fig = px.treemap(
        df_all,
        path=['status', 'label'],
        values='count',
        color='status',
        color_discrete_map={'Approved': '#c6c0cf', 'Rejected': '#716287'},
        title="Top Words in Loan Purpose (Normalized by Approval State)"
    )

    # Mostrar porcentaje como etiqueta
    fig.update_traces(texttemplate='%{label}',)

    fig.update_layout(width=1100, height=500)

    fig.show()


# Load data

In [0]:
#df_loan = pd.read_csv('/Volumes/addi_prod/ds/ds_volume/files/datascience/co/nata_files/Loan_status_2007-2020Q3.gzip.zip').drop(['Unnamed: 0'], axis=1)


#print('Data dimension', df_loan.shape)

#df_loan = df_loan[df_loan['loan_status'].notna()]

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

df_loan_0 = pd.read_csv('nata_files/accepted_2007_to_2018Q4.csv.gz')

df_loan_1 = pd.read_csv('nata_files/rejected_2007_to_2018Q4.csv.gz')

print('Dimension accepted', df_loan_0.shape)

print('Dimension rejected', df_loan_1.shape)

df_loan_0 = df_loan_0[0:2260698]

/home/spark-5cc75cd9-d898-47a9-8222-52/.ipykernel/117014/command-5567060867414662-3253906041:10: DtypeWarning:

Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.



Dimension accepted (2260701, 151)
Dimension rejected (27648741, 9)


#Approval rate

In [0]:
df_loan_1 = df_loan_1.rename(columns={'Amount Requested':'loan_amnt', 'Application Date':'issue_d', 'Debt-To-Income Ratio':'dti', 'Zip Code':'zip_code', 'Employment Length':'emp_length', 'Policy Code':'policy_code', 'Loan Title': 'title', 'State':'addr_state'})

df_loan_0['app_date'] = pd.to_datetime(df_loan_0['issue_d'], format='%b-%Y')

df_loan_1['issue_d'] = pd.to_datetime(df_loan_1['issue_d'])

df_loan_1['app_date'] =  df_loan_1['issue_d'].dt.to_period('M').dt.to_timestamp()

df_loan_1 = df_loan_1.reset_index().rename(columns={'index':'id'})


#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

cols_to_select = ['app_date',  'approved', 'id', 'loan_amnt', 'dti', 'zip_code','emp_length', 'policy_code' , 'addr_state', 'title']

df_loan_0['approved'] = 1

df_loan_1['approved'] = 0

df_loan_1['dti'] = np.round(df_loan_1['dti'].str.replace('%', '', regex=False).astype(float),2)


df_loan_total = pd.concat([df_loan_0[cols_to_select], df_loan_1[cols_to_select]], axis=0)

df_approved = df_loan_total.groupby(['app_date']).agg({'approved':'mean', 'id':'count'}).reset_index()

df_approved['approved'] = df_approved['approved']*100


In [0]:

plot_chart(
    df=df_approved,
    chart_type="bar",
    x_col="app_date",
    y_col="id",
    text_col="",
    sort_by="app_date",
    color_sequence=['#b4a7d6'],
    ascending=False,
    x_label="Application Date",
    y_label="Number of Applications",
    title="Number of credit applications between 2007 and 2018",
    width=1200,
    height=500,
)


In [0]:
plot_chart(
    df=df_approved,
    chart_type="line",
    x_col="app_date",
    y_col="approved",
    text_col="",
    sort_by="app_date",
    ascending=False,
    color_sequence=['#b4a7d6'],
    x_label="Application Date",
    y_label="Percentage of approved loans (%)",
    title="Approval rate of credit applications between 2007 and 2018",
    width=1200,
    height=500,
)


# Who are the rejecteds?

## Analysis by Risk Segment

In [0]:
df_loan_1['Risk_Score'] = df_loan_1['Risk_Score'].fillna(100000)

df_loan_1['score_bins'] = pd.cut(df_loan_1['Risk_Score'], bins=[0,600,800,1000, 100000], labels=['0-600','600-800','800-1000', 'NA'])

df_loan_1['quarter_date'] = df_loan_1['app_date'].dt.to_period('Q').dt.to_timestamp()





df_score_dist = df_loan_1.groupby(['quarter_date','score_bins']).agg({'id':'count'}).reset_index()

df_score_dist['pct_score_bins'] = df_score_dist['id']/df_score_dist.groupby('quarter_date')['id'].transform('sum')

df_score_dist = df_score_dist.rename(columns={'score_bins':'Risk Score Segment'})

df_score_dist['pct_score_bins'] = df_score_dist['pct_score_bins']*100

In [0]:
plot_chart(
    df= df_score_dist, 
    chart_type="line",
    x_col="quarter_date",
    y_col="pct_score_bins",
    group_col="Risk Score Segment",
    text_col="",
    color_sequence= [ '#d2cae6', '#b4a7d6', '#ae73be', 'black', ],
    text_as_percentage=True,
    text_position="top center",
    x_label='Application Date',
    y_label='%Rejected Loans',
    title="Rejection Rate Distribution Across Risk Score Segments",
    width=1200,
    height=400,
)

## Analysis by Risk Segments and Debt-To-Income Ratio

In [0]:
df_loan_1['dti_segments'] = pd.cut(df_loan_1['dti'], bins =[-10, 20, 40, 60, 80, 100, 100000000000], labels=['0-20%', '20-40%', '40-60%', '60-80%', '80-100%', '100%+'])

df_group = df_loan_1.groupby(['quarter_date', 'dti_segments']).agg({'id':'count', 'loan_amnt':'mean'}).reset_index()

df_group['pct_score_dti_bins'] = df_group['id']/df_group.groupby('quarter_date')['id'].transform('sum')

df_group['pct_score_dti_bins'] = df_group['pct_score_dti_bins']*100

df_group = df_group.rename(columns={'dti_segments':'DTI Segments'})


plot_chart(
    df= df_group, 
    chart_type="line",
    x_col="quarter_date",
    y_col="pct_score_dti_bins",
    group_col="DTI Segments",
    text_col="",
    color_sequence= [ '#d2cae6', '#aaa2ba', '#ae73be', '#795085', 'black', '#ff7f7f' ],
    text_as_percentage=True,
    text_position="top center",
    x_label='Application Date',
    y_label='%Rejected Loans',
    title="Rejection Rate Distribution Across Debt to Income Ratio Segments",
    width=1200,
    height=400,
)

In [0]:


df_loan_1['dti_segments'] = pd.cut(df_loan_1['dti'], bins =[-10, 20, 40, 60, 80, 100, 100000000000], labels=['0-20%', '20-40%', '40-60%', '60-80%', '80-100%', '100%+'])

df_group = df_loan_1.groupby(['score_bins', 'dti_segments']).agg({'id':'count', 'loan_amnt':'mean'}).reset_index()

df_group['pct_score_dti_bins'] = df_group['id']/df_group.groupby('score_bins')['id'].transform('sum')

df_group['pct_score_dti_bins'] = df_group['pct_score_dti_bins']*100

df_group = df_group.rename(columns={'dti_segments':'Debt to Income Ratio Segments'})


#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

plot_chart(
    df= df_group, 
    chart_type="bar",
    x_col="score_bins",
    y_col="pct_score_dti_bins",
    group_col="Debt to Income Ratio Segments",
    text_col="",
    color_sequence=['#e1dbee', '#d2cae6', '#b4a7d6', '#9085ab', '#5a536b', '#363140', 'black'],
    text_as_percentage=True,
    text_position="top center",
    x_label='Risk Score Segment',
    y_label='%Rejected Loans',
    title="Rejection Rate Distribution by Debt-to-Income and Risk Score Segments",
    width=1200,
    height=400,
)

/databricks/python/lib/python3.11/site-packages/plotly/express/_core.py:1958: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/databricks/python/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## Amount

In [0]:
bins = [0, 2000, 5000, 10000, 20000, float('inf')]


labels = [
    "0–2,000", 
    "2,001–5,000", 
    "5,001–10,000", 
    "10,001–20,000", 
    "20,001+", 
]

df_loan_1["amount_segment"] = pd.cut(df_loan_1["loan_amnt"], bins=bins, labels=labels, include_lowest=True)


df_group = df_loan_1.groupby(['quarter_date', 'amount_segment']).agg({'id':'count'}).reset_index()

df_group['pct_score_dti_bins'] = df_group['id']/df_group.groupby('quarter_date')['id'].transform('sum')

df_group['pct_score_dti_bins'] = df_group['pct_score_dti_bins']*100

df_group = df_group.rename(columns={'amount_segment':'Requested Amount Segments'})


plot_chart(
    df= df_group, 
    chart_type="line",
    x_col="quarter_date",
    y_col="pct_score_dti_bins",
    group_col="Requested Amount Segments",
    text_col="",
    color_sequence= [ '#d2cae6', '#aaa2ba', '#ae73be', '#795085', 'black', '#ff7f7f' ],
    text_as_percentage=True,
    text_position="top center",
    x_label='Application Date',
    y_label='%Rejected Loans',
    title="Rejection Rate Distribution Across Debt to Income Ratio Segments",
    width=1200,
    height=400,
)

In [0]:
custom_colorscale = [
    [0.0, "#d2cae6"],   # más claro
    [0.2, "#bbafda"],   
    [0.4, "#a296c0"],   
    [0.6, "#8b77a3"],   
    [0.8, "#716287"],   
    [1.0, "#5a536b"]    # morado profundo
]


heatmap_categoricas_vs_numerica(df_group, 'Debt to Income Ratio Segments', 'score_bins', 'loan_amnt',   colorscale= custom_colorscale, width=1000, height=400, title='Average Requested Amount by Risk Score and Debt-to-Income Segments', x_title='Debt to Income Ratio', y_title='Risk Score')


# Rejected VS Approved

In [0]:
def segment_employment_length(emp_length):
    if emp_length == "< 1 year":
        return "Less than 1 year"
    elif emp_length in ["1 year", "2 years", "3 years", "4 years", "5 years", "6 years"]:
        return "1–6 years"
    elif emp_length in ["7 years", "8 years", "9 years", "10+ years"]:
        return "7+ years"
    else:
        return "Unknown"

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

df_loan_total["employment_segment"] = df_loan_total["emp_length"].apply(segment_employment_length)

df_loan_total['dti_segments'] = pd.cut(df_loan_total['dti'], bins =[-100, 20.0, 40.0, 60.0, 80.0, 100.0, 100000000000], labels=['0-20%', '20-40%', '40-60%', '60-80%', '80-100%', '100%+'])

bins = [0, 2000, 5000, 10000, 20000, 40000, float('inf')]

labels = [
    "0–2,000", 
    "2,001–5,000", 
    "5,001–10,000", 
    "10,001–20,000", 
    "20,001–40,000", 
    "40,001+", 
]

df_loan_total["amount_segment"] = pd.cut(df_loan_total["loan_amnt"], bins=bins, labels=labels, include_lowest=True)

df_group = df_loan_total.groupby(['approved', 'amount_segment']).agg({'id':'count'}).reset_index()

df_group['pct_amt_bins'] = np.round((df_group['id']/df_group.groupby('approved')['id'].transform('sum'))*100,2)

df_group['approved'] = np.where(df_group['approved']==1, 'Yes', 'No')

df_group = df_group.rename(columns={'approved':'Did the application get approved?'})

## Amount

In [0]:
plot_chart(
    df= df_group, 
    chart_type="bar",
    x_col="amount_segment",
    y_col="pct_amt_bins",
    group_col="Did the application get approved?",
    text_col="",
    color_sequence=[ '#9085ab','#e1dbee' ],
    text_as_percentage=True,
    text_position="top center",
    x_label='Amount Segments',
    y_label='%Apps',
    title="Application Distribution by Approval Status and Amount Segments",
    width=1200,
    height=400,
)

## Location

In [0]:
df_group = df_loan_total.groupby(['approved', 'addr_state']).agg({'id':'count'}).reset_index()

df_group['pct_amt_bins'] = np.round((df_group['id']/df_group.groupby('approved')['id'].transform('sum'))*100,2)

df_group['approved'] = np.where(df_group['approved']==1, 'Yes', 'No')

df_group = df_group.rename(columns={'approved':'Did the application get approved?'})


#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
plot_chart(
    df= df_group.sort_values(by='pct_amt_bins', ascending=False), 
    chart_type="bar",
    x_col="addr_state",
    y_col="pct_amt_bins",
    group_col="Did the application get approved?",
    text_col="",
    color_sequence=[ '#e1dbee', '#9085ab'],
    text_as_percentage=True,
    text_position="top center",
    x_label='State (US)',
    y_label='%Apps',
    title="Application Distribution by Approval Status and State (US)",
    width=1200,
    height=400,
)

##Empl. Length

In [0]:
df_group = df_loan_total.groupby(['approved', 'employment_segment']).agg({'id':'count'}).reset_index()

df_group['pct_amt_bins'] = np.round((df_group['id']/df_group.groupby('approved')['id'].transform('sum'))*100,2)

df_group['approved'] = np.where(df_group['approved']==1, 'Yes', 'No')

df_group = df_group.rename(columns={'approved':'Did the application get approved?'})


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

plot_chart(
    df= df_group, 
    chart_type="bar",
    x_col="employment_segment",
    y_col="pct_amt_bins",
    group_col="Did the application get approved?",
    text_col="",
    color_sequence=[  '#9085ab', '#e1dbee',],
    text_as_percentage=True,
    text_position="top center",
    x_label='Client Employment Length',
    y_label='%Apps',
    title="Application Distribution by Approval Status and Client Employment Length",
    width=1200,
    height=400,
    category_order=['Less than 1 year', "1–6 years", '7+ years', 'Unknown']
)

## DTI and Amount segments

In [0]:
df_group = df_loan_total.groupby(['amount_segment', 'dti_segments']).agg({'approved':'mean'}).reset_index()

df_group['approved'] = df_group['approved']*100

custom_colorscale = [
    [0.0, "#d3bfe6"],   # más claro
    [0.2, "#bca7d8"],   
    [0.4, "#a38ec0"],   
    [0.6, "#8b77a3"],   
    [0.8, "#716287"],   
    [1.0, "#5a536b"]    # morado profundo
]

heatmap_categoricas_vs_numerica(df_group, 'amount_segment', 'dti_segments', 'approved',   colorscale= custom_colorscale, width=1000, height=500, title='%Approval Status by Requested amount and Debt-to-Income Segments', x_title='Requested amount Segments', y_title='Debt to income Ratio')


## Word cloud

In [0]:
nltk.download('stopwords')
english_stopwords = set(stopwords.words('english'))


custom_stopwords = english_stopwords.union({
    'loan', 'credit', 'use', 'get', 'make', 'pay', 'need', 'want', 'help', 'due', 'debt', 'consolidation', 'small', 'financing', 'improvement', 'purchase'
})

plot_purpose_treemap(df_loan_total, status_column='approved', purpose_column='title')


IOStream.flush timed out
